In [34]:
remote_timing_file = "http://portal.nersc.gov/project/astro250/xswap/iptf/timing.csv"
remote_image_file = "http://portal.nersc.gov/project/astro250/xswap/iptf/image.csv"
remote_readme_file = "http://portal.nersc.gov/project/astro250/xswap/iptf/README"

# Please set local storage for the files
target_dir = "~/tmp/iptf"
!mkdir -p $target_dir;

local_timing_file = target_dir + "/timing.csv"
local_image_file = target_dir + "/image.csv"
local_readme_file = target_dir + "/README"
sorted_image_file = target_dir + "/sorted_image.csv"
sorted_timing_file = target_dir + "/sorted_timing.csv"

import os.path
if not os.path.exists(local_readme_file):
    print("Downloading data. It may take a while depending on your network connection.")
    !cd $target_dir;wget $remote_timing_file
    !cd $target_dir;wget $remote_image_file
    !cd $target_dir;wget $remote_readme_file

!cd $target_dir;cat README

--2018-08-01 14:27:31--  http://portal.nersc.gov/project/astro250/xswap/iptf/timing.csv
Resolving portal.nersc.gov (portal.nersc.gov)... 128.55.201.128
Connecting to portal.nersc.gov (portal.nersc.gov)|128.55.201.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1120623419 (1.0G) [text/csv]
Saving to: ‘timing.csv’

timing.csv          100%[===================>]   1.04G  21.9MB/s    in 47s     

2018-08-01 14:28:18 (22.6 MB/s) - ‘timing.csv’ saved [1120623419/1120623419]

--2018-08-01 14:28:18--  http://portal.nersc.gov/project/astro250/xswap/iptf/image.csv
Resolving portal.nersc.gov (portal.nersc.gov)... 128.55.201.128
Connecting to portal.nersc.gov (portal.nersc.gov)|128.55.201.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 96224615 (92M) [text/csv]
Saving to: ‘image.csv’

image.csv           100%[===================>]  91.77M  22.5MB/s    in 4.3s    

2018-08-01 14:28:22 (21.4 MB/s) - ‘image.csv’ saved [96224615/96224615]

--20

In [259]:
import pandas as pd
import datetime

def get_timing_features():
    time_features = []
    utime_features = []
    for i in range(0,40):
        time_features.append('time_' + str(i))
        utime_features.append('utime_' + str(i))
    return time_features, utime_features



def loadImageFile (image_file, timing_file):
    print ("reading " + image_file)
    images = pd.read_csv(image_file, index_col = 0)
    time_features, utime_features = get_timing_features()
    features = time_features + utime_features
    tf = {}
    
    for x in features:
        images.insert(loc=len(images.keys()), column = x, value = None)
        tf[x] = []
   
    print ("reading " + timing_file)
    timings = pd.read_csv(timing_file) 
    print ("re-indexing")
    timings.reset_index(inplace=True)
    print ("merging files")
    t_index = 0
    t_len = len(timings)
    while t_index < t_len:
        for cid in range(40):
            cp = timings.iloc[t_index + cid]
            cp_id = cp['id']
            cp_cid = cp['checkpointid']
            cp_time = cp['time']
            cp_utime = cp['utime']
            name = 'time_' + str(cp_cid)
            tf[name].append(cp_time)
            name = 'utime_' + str(cp_cid)
            tf[name].append(cp_utime)    
            if (t_index + cid) % 100000 == 0:
                print(str(t_index) + " of " + str(t_len) +"\t\t"+ str(datetime.datetime.now()))
        t_index += 40
        
    merge_files(images, tf)
    
    return images, timings, tf


def merge_files(images, tf):
    for key, values in tf.items():
        images[key].replace(images[key].tolist(), pd.Series(values), inplace=True) 



def normalize_file(src_file, dst_file):
    print ('load ' + src_file)
    timings = pd.read_csv(src_file)
    print ('sort id' + src_file)
    sorted_timings = timings.sort_values(by='id')
    print ("write " + dst_file)
    sorted_timings.to_csv(path_or_buf=dst_file, index=False)
    print ("completed")


In [151]:
   z = timings[['checkpointid','time','utime']]
    nb = len(z)
    i = 0
    j = 0
    while j < nb:
        x= ((z[j:j+40]).sort_values(by='checkpointid'))[['time','utime']]
        rt = pd.DataFrame(x['time'])
        rt.index = t
        ru = pd.DataFrame(x['utime'])
        ru = ru.rename (columns= {"utime":"time"})
        ru.index = u
        rr = rt.append(ru, sort=False)
        rr = rr.T 
        #print (rr.iloc[0])
        images.iloc[i,18:] = rr.iloc[0]
        #print (images.iloc[i])
        j += 40
        i += 1
        if i % 1000:
            print(str + datetime.datetime.now())

IndentationError: unexpected indent (<ipython-input-151-58d08298efd4>, line 2)

In [39]:
normalize_file(local_image_file, sorted_image_file)
normalize_file(local_timing_file, sorted_timing_file)

load ~/tmp/iptf/image.csv
sort id~/tmp/iptf/image.csv
write ~/tmp/iptf/sorted_image.csv
load ~/tmp/iptf/timing.csv
sort id~/tmp/iptf/timing.csv
write ~/tmp/iptf/sorted_timing.csv


In [223]:
images=None
timings=None

images, timings, tf = loadImageFile(sorted_image_file, sorted_timing_file)

reading ~/tmp/iptf/sorted_image.csv
reading ~/tmp/iptf/sorted_timing.csv
re-indexing
merging files
0 of 24019080		2018-08-02 10:11:02.414470
10000 of 24019080		2018-08-02 10:11:04.558130
20000 of 24019080		2018-08-02 10:11:06.728249
30000 of 24019080		2018-08-02 10:11:08.790627
40000 of 24019080		2018-08-02 10:11:11.029776
50000 of 24019080		2018-08-02 10:11:13.227219
60000 of 24019080		2018-08-02 10:11:15.337378
70000 of 24019080		2018-08-02 10:11:17.381922
80000 of 24019080		2018-08-02 10:11:19.650311
90000 of 24019080		2018-08-02 10:11:21.646578
100000 of 24019080		2018-08-02 10:11:23.681164
110000 of 24019080		2018-08-02 10:11:25.765697
120000 of 24019080		2018-08-02 10:11:27.814986
130000 of 24019080		2018-08-02 10:11:29.803110
140000 of 24019080		2018-08-02 10:11:31.782908
150000 of 24019080		2018-08-02 10:11:33.753977
160000 of 24019080		2018-08-02 10:11:35.891732
170000 of 24019080		2018-08-02 10:11:37.965031
180000 of 24019080		2018-08-02 10:11:40.012000
190000 of 24019080		20

1710000 of 24019080		2018-08-02 10:17:03.106574
1720000 of 24019080		2018-08-02 10:17:05.239490
1730000 of 24019080		2018-08-02 10:17:07.252951
1740000 of 24019080		2018-08-02 10:17:09.302364
1750000 of 24019080		2018-08-02 10:17:11.275674
1760000 of 24019080		2018-08-02 10:17:13.294359
1770000 of 24019080		2018-08-02 10:17:15.372853
1780000 of 24019080		2018-08-02 10:17:17.660555
1790000 of 24019080		2018-08-02 10:17:19.742320
1800000 of 24019080		2018-08-02 10:17:21.859778
1810000 of 24019080		2018-08-02 10:17:24.046130
1820000 of 24019080		2018-08-02 10:17:26.250918
1830000 of 24019080		2018-08-02 10:17:28.286544
1840000 of 24019080		2018-08-02 10:17:30.288912
1850000 of 24019080		2018-08-02 10:17:32.278118
1860000 of 24019080		2018-08-02 10:17:34.260166
1870000 of 24019080		2018-08-02 10:17:36.273233
1880000 of 24019080		2018-08-02 10:17:38.284302
1890000 of 24019080		2018-08-02 10:17:40.442113
1900000 of 24019080		2018-08-02 10:17:42.475849
1910000 of 24019080		2018-08-02 10:17:44

3420000 of 24019080		2018-08-02 10:24:02.708931
3430000 of 24019080		2018-08-02 10:24:04.968799
3440000 of 24019080		2018-08-02 10:24:07.511017
3450000 of 24019080		2018-08-02 10:24:09.759705
3460000 of 24019080		2018-08-02 10:24:12.101158
3470000 of 24019080		2018-08-02 10:24:14.326939
3480000 of 24019080		2018-08-02 10:24:16.626289
3490000 of 24019080		2018-08-02 10:24:19.015688
3500000 of 24019080		2018-08-02 10:24:21.368529
3510000 of 24019080		2018-08-02 10:24:23.821552
3520000 of 24019080		2018-08-02 10:24:26.063230
3530000 of 24019080		2018-08-02 10:24:28.599306
3540000 of 24019080		2018-08-02 10:24:30.933619
3550000 of 24019080		2018-08-02 10:24:33.321667
3560000 of 24019080		2018-08-02 10:24:35.369155
3570000 of 24019080		2018-08-02 10:24:37.499511
3580000 of 24019080		2018-08-02 10:24:39.706511
3590000 of 24019080		2018-08-02 10:24:41.741016
3600000 of 24019080		2018-08-02 10:24:43.766155
3610000 of 24019080		2018-08-02 10:24:45.794994
3620000 of 24019080		2018-08-02 10:24:47

5130000 of 24019080		2018-08-02 10:30:18.418645
5140000 of 24019080		2018-08-02 10:30:20.414626
5150000 of 24019080		2018-08-02 10:30:22.455188
5160000 of 24019080		2018-08-02 10:30:24.445381
5170000 of 24019080		2018-08-02 10:30:26.444744
5180000 of 24019080		2018-08-02 10:30:28.470533
5190000 of 24019080		2018-08-02 10:30:30.487157
5200000 of 24019080		2018-08-02 10:30:32.511205
5210000 of 24019080		2018-08-02 10:30:34.525670
5220000 of 24019080		2018-08-02 10:30:36.532716
5230000 of 24019080		2018-08-02 10:30:38.517758
5240000 of 24019080		2018-08-02 10:30:40.543140
5250000 of 24019080		2018-08-02 10:30:42.580829
5260000 of 24019080		2018-08-02 10:30:44.559863
5270000 of 24019080		2018-08-02 10:30:46.588223
5280000 of 24019080		2018-08-02 10:30:48.643072
5290000 of 24019080		2018-08-02 10:30:50.630865
5300000 of 24019080		2018-08-02 10:30:52.669901
5310000 of 24019080		2018-08-02 10:30:54.664345
5320000 of 24019080		2018-08-02 10:30:56.745126
5330000 of 24019080		2018-08-02 10:30:58

6840000 of 24019080		2018-08-02 10:36:26.075121
6850000 of 24019080		2018-08-02 10:36:28.066877
6860000 of 24019080		2018-08-02 10:36:30.094056
6870000 of 24019080		2018-08-02 10:36:32.115326
6880000 of 24019080		2018-08-02 10:36:34.113479
6890000 of 24019080		2018-08-02 10:36:36.123430
6900000 of 24019080		2018-08-02 10:36:38.106489
6910000 of 24019080		2018-08-02 10:36:40.118502
6920000 of 24019080		2018-08-02 10:36:42.172264
6930000 of 24019080		2018-08-02 10:36:44.189573
6940000 of 24019080		2018-08-02 10:36:46.205705
6950000 of 24019080		2018-08-02 10:36:48.212938
6960000 of 24019080		2018-08-02 10:36:50.230886
6970000 of 24019080		2018-08-02 10:36:52.255915
6980000 of 24019080		2018-08-02 10:36:54.266339
6990000 of 24019080		2018-08-02 10:36:56.279418
7000000 of 24019080		2018-08-02 10:36:58.312528
7010000 of 24019080		2018-08-02 10:37:00.426433
7020000 of 24019080		2018-08-02 10:37:02.524722
7030000 of 24019080		2018-08-02 10:37:04.620695
7040000 of 24019080		2018-08-02 10:37:06

...

16980000 of 24019080		2018-08-02 11:13:53.597863
16990000 of 24019080		2018-08-02 11:13:55.689388
17000000 of 24019080		2018-08-02 11:13:57.792841
17010000 of 24019080		2018-08-02 11:13:59.831539
17020000 of 24019080		2018-08-02 11:14:01.951766
17030000 of 24019080		2018-08-02 11:14:04.007562
17040000 of 24019080		2018-08-02 11:14:06.638768
17050000 of 24019080		2018-08-02 11:14:09.100687
17060000 of 24019080		2018-08-02 11:14:11.712128
17070000 of 24019080		2018-08-02 11:14:14.214338
17080000 of 24019080		2018-08-02 11:14:16.881392
17090000 of 24019080		2018-08-02 11:14:19.657011
17100000 of 24019080		2018-08-02 11:14:22.517918
17110000 of 24019080		2018-08-02 11:14:25.387311
17120000 of 24019080		2018-08-02 11:14:28.221533
17130000 of 24019080		2018-08-02 11:14:31.066231
17140000 of 24019080		2018-08-02 11:14:33.836472
17150000 of 24019080		2018-08-02 11:14:36.724851
17160000 of 24019080		2018-08-02 11:14:39.532686
17170000 of 24019080		2018-08-02 11:14:41.884162
17180000 of 24019080

18660000 of 24019080		2018-08-02 11:19:53.899533
18670000 of 24019080		2018-08-02 11:19:55.971093
18680000 of 24019080		2018-08-02 11:19:58.003610
18690000 of 24019080		2018-08-02 11:20:00.045955
18700000 of 24019080		2018-08-02 11:20:02.131223
18710000 of 24019080		2018-08-02 11:20:04.140113
18720000 of 24019080		2018-08-02 11:20:06.201919
18730000 of 24019080		2018-08-02 11:20:08.278601
18740000 of 24019080		2018-08-02 11:20:10.347521
18750000 of 24019080		2018-08-02 11:20:12.428390
18760000 of 24019080		2018-08-02 11:20:14.529296
18770000 of 24019080		2018-08-02 11:20:16.568680
18780000 of 24019080		2018-08-02 11:20:18.623078
18790000 of 24019080		2018-08-02 11:20:20.671155
18800000 of 24019080		2018-08-02 11:20:22.785009
18810000 of 24019080		2018-08-02 11:20:24.853467
18820000 of 24019080		2018-08-02 11:20:26.905748
18830000 of 24019080		2018-08-02 11:20:28.988933
18840000 of 24019080		2018-08-02 11:20:31.222685
18850000 of 24019080		2018-08-02 11:20:33.299086
18860000 of 24019080

20340000 of 24019080		2018-08-02 11:26:07.698551
20350000 of 24019080		2018-08-02 11:26:09.778938
20360000 of 24019080		2018-08-02 11:26:11.770597
20370000 of 24019080		2018-08-02 11:26:13.785328
20380000 of 24019080		2018-08-02 11:26:15.788779
20390000 of 24019080		2018-08-02 11:26:17.793510
20400000 of 24019080		2018-08-02 11:26:19.768725
20410000 of 24019080		2018-08-02 11:26:21.799389
20420000 of 24019080		2018-08-02 11:26:23.788702
20430000 of 24019080		2018-08-02 11:26:25.764040
20440000 of 24019080		2018-08-02 11:26:27.767211
20450000 of 24019080		2018-08-02 11:26:29.784815
20460000 of 24019080		2018-08-02 11:26:31.826612
20470000 of 24019080		2018-08-02 11:26:33.882548
20480000 of 24019080		2018-08-02 11:26:35.906084
20490000 of 24019080		2018-08-02 11:26:37.956958
20500000 of 24019080		2018-08-02 11:26:39.970035
20510000 of 24019080		2018-08-02 11:26:42.004835
20520000 of 24019080		2018-08-02 11:26:44.046144
20530000 of 24019080		2018-08-02 11:26:46.088925
20540000 of 24019080

22020000 of 24019080		2018-08-02 11:31:47.354328
22030000 of 24019080		2018-08-02 11:31:49.382581
22040000 of 24019080		2018-08-02 11:31:51.412371
22050000 of 24019080		2018-08-02 11:31:53.433575
22060000 of 24019080		2018-08-02 11:31:55.440877
22070000 of 24019080		2018-08-02 11:31:57.482056
22080000 of 24019080		2018-08-02 11:31:59.455009
22090000 of 24019080		2018-08-02 11:32:01.428839
22100000 of 24019080		2018-08-02 11:32:03.450551
22110000 of 24019080		2018-08-02 11:32:05.451871
22120000 of 24019080		2018-08-02 11:32:07.462954
22130000 of 24019080		2018-08-02 11:32:09.519184
22140000 of 24019080		2018-08-02 11:32:11.523358
22150000 of 24019080		2018-08-02 11:32:13.548792
22160000 of 24019080		2018-08-02 11:32:15.576229
22170000 of 24019080		2018-08-02 11:32:17.572002
22180000 of 24019080		2018-08-02 11:32:19.568579
22190000 of 24019080		2018-08-02 11:32:21.555809
22200000 of 24019080		2018-08-02 11:32:23.546099
22210000 of 24019080		2018-08-02 11:32:25.561201
22220000 of 24019080

23700000 of 24019080		2018-08-02 11:37:50.218336
23710000 of 24019080		2018-08-02 11:37:52.241777
23720000 of 24019080		2018-08-02 11:37:54.223531
23730000 of 24019080		2018-08-02 11:37:56.270359
23740000 of 24019080		2018-08-02 11:37:58.262854
23750000 of 24019080		2018-08-02 11:38:00.286088
23760000 of 24019080		2018-08-02 11:38:02.329488
23770000 of 24019080		2018-08-02 11:38:04.368790
23780000 of 24019080		2018-08-02 11:38:06.443376
23790000 of 24019080		2018-08-02 11:38:08.443972
23800000 of 24019080		2018-08-02 11:38:10.466644
23810000 of 24019080		2018-08-02 11:38:12.499425
23820000 of 24019080		2018-08-02 11:38:14.484537
23830000 of 24019080		2018-08-02 11:38:16.521997
23840000 of 24019080		2018-08-02 11:38:18.499177
23850000 of 24019080		2018-08-02 11:38:20.482370
23860000 of 24019080		2018-08-02 11:38:22.515880
23870000 of 24019080		2018-08-02 11:38:24.887161
23880000 of 24019080		2018-08-02 11:38:26.907935
23890000 of 24019080		2018-08-02 11:38:29.020188
23900000 of 24019080

In [ ]:
merge_files(images, tf)

In [1]:
2

2

In [2]:
len(tf)

NameError: name 'tf' is not defined

In [3]:
images

NameError: name 'images' is not defined

transfer time vs time of night

utc_obs vs. time between step 0 and 1

Galactic latitude vs subtraction time:

gal_lat vs. time between 24 and 26

In [103]:
df = pd.DataFrame(data=[[1,2]],columns = ['a','b'])

In [255]:
z = timings[['checkpointid','time','utime']]

In [272]:
x= ((z[0:40]).sort_values(by='checkpointid'))[['time','utime']]

In [273]:
print(x)

                                time        utime
23423596     2015-03-20 03:04:37.702     0.000000
22818126  2015-03-20 04:17:12.423566  4354.721566
22217649  2015-03-20 04:17:12.506983     0.083417
21617172  2015-03-20 04:17:13.374036     0.867053
21016695  2015-03-20 04:17:13.745572     0.371536
20416218  2015-03-20 04:17:15.403216     1.657644
19815741  2015-03-20 04:17:15.542033     0.138817
19215264  2015-03-20 04:17:17.766029     2.223996
18614787  2015-03-20 04:17:18.524554     0.758525
18014310  2015-03-20 04:17:38.965292    20.440738
17413833  2015-03-20 04:17:39.001262     0.035970
16813356  2015-03-20 04:17:39.085677     0.084415
16212879  2015-03-20 04:17:41.256771     2.171094
15612402  2015-03-20 04:17:43.001172     1.744401
15011925  2015-03-20 04:17:47.205149     4.203977
14411448  2015-03-20 04:17:47.386064     0.180915
13810971  2015-03-20 04:17:47.773047     0.386983
13210494  2015-03-20 04:17:47.816621     0.043574
12610017  2015-03-20 04:18:08.843939    21.027318


In [393]:
t,u = get_timing_features()
rt = pd.DataFrame(x['time'])
rt.index = t
ru = pd.DataFrame(x['utime'])
ru = ru.rename (columns= {"utime":"time"})
ru.index = u
rr = rt.append(ru, sort=False)
rr = rr.T

In [394]:
print (rr)

                       time_0                      time_1  \
time  2015-03-20 03:04:37.702  2015-03-20 04:17:12.423566   

                          time_2                      time_3  \
time  2015-03-20 04:17:12.506983  2015-03-20 04:17:13.374036   

                          time_4                      time_5  \
time  2015-03-20 04:17:13.745572  2015-03-20 04:17:15.403216   

                          time_6                      time_7  \
time  2015-03-20 04:17:15.542033  2015-03-20 04:17:17.766029   

                          time_8                      time_9   ...     \
time  2015-03-20 04:17:18.524554  2015-03-20 04:17:38.965292   ...      

     utime_30 utime_31 utime_32 utime_33 utime_34 utime_35 utime_36 utime_37  \
time  13.7295   2.4327  35.4989   2.2319  8.63361  5.97557  2.77407  120.336   

      utime_38 utime_39  
time  0.241639  1.34092  

[1 rows x 80 columns]


In [388]:
print(len(rr))

41


In [107]:
d1 = pd.DataFrame([[1,2,3],[4,5,6]], index=[6,7],columns=['a','b','c'])

In [414]:
d2 = pd.DataFrame([[10]],columns=[1])

In [416]:
dx = pd.concat([d1,d2],axis=1)
dx

,0,1,1
0,1,2,10.0
1,3,4,NaN


In [110]:
d1.append(pd.DataFrame([{'a':10,'b':11,'c':12}]))

,a,b,c
6,1,2,3
7,4,5,6
0,10,11,12


In [251]:
d1

,a,b,c
6,1,2,3
7,4,5,6


In [256]:
#dc = pd.DataFrame([101,102], columns=['c'])
#dc
d1['c'].replace(d1['c'].tolist(),pd.Series([101,102]))

6    101
7    102
Name: c, dtype: int64

In [102]:
r = d1.iloc[0,0:]

In [254]:
dir(d1['c'])

['T',
 '_AXIS_ALIASES',
 '_AXIS_IALIASES',
 '_AXIS_LEN',
 '_AXIS_NAMES',
 '_AXIS_NUMBERS',
 '_AXIS_ORDERS',
 '_AXIS_REVERSED',
 '_AXIS_SLICEMAP',
 '__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_prepare__',
 '__array_priority__',
 '__array_wrap__',
 '__bool__',
 '__bytes__',
 '__class__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__div__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__finalize__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__imod__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__long__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 

In [248]:
d1['c']

6    3
7    6
Name: c, dtype: int64